O Princípio da Responsabilidade Única (SRP) é o primeiro princípio do SOLID. Ele afirma que uma classe deve ter apenas uma razão para mudar, ou seja, ela deve ter apenas uma responsabilidade.



Classe abstrata

In [1]:
abstract class Investimento {
    constructor(public nome: string, public valorInvestido: number) {}

    abstract calcularRetornoAnual(): number;

    descrever(): string {
        return `Investimento: ${this.nome}, Valor Investido: R$ ${this.valorInvestido}`;
    }
}

Extends investimento

In [2]:
class Acao extends Investimento {
    constructor(
        nome: string,
        valorInvestido: number,
        public quantidade: number,
        public precoPorAcao: number,
        public dividendoAnual: number
    ) {
        super(nome, valorInvestido);
    }

    calcularRetornoAnual(): number {
        return this.quantidade * this.dividendoAnual;
    }

    descrever(): string {
        return `${super.descrever()}, Quantidade: ${this.quantidade}, Preço por Ação: R$ ${this.precoPorAcao}, Dividendo Anual: R$ ${this.dividendoAnual}`;
    }
}

Acao Service (Violação)

In [3]:
class AcaoService {
    calcularRetornoAnual(acao: Acao): number {
        return acao.quantidade * acao.dividendoAnual;
    }

    descrever(acao: Acao): string {
        return `Ação: ${acao.nome}, Quantidade: ${acao.quantidade}, Preço por Ação: R$ ${acao.precoPorAcao}, Dividendo Anual: R$ ${acao.dividendoAnual}`;
    }

    save(acao: Acao): void {
        console.log(`Saving investment: ${JSON.stringify(acao)}`);
    }
}

Executando

In [4]:
const acao = new Acao("Empresa X", 5000, 100, 50, 5);
const acaoService = new AcaoService();
console.log(acaoService.descrever(acao));


Ação: Empresa X, Quantidade: 100, Preço por Ação: R$ 50, Dividendo Anual: R$ 5


In [5]:
console.log(`Retorno anual: R$ ${acaoService.calcularRetornoAnual(acao)}`);


Retorno anual: R$ 500


In [6]:
acaoService.save(acao);

Saving investment: {"nome":"Empresa X","valorInvestido":5000,"quantidade":100,"precoPorAcao":50,"dividendoAnual":5}


Refatorando o código (Service)

In [7]:
class AcaoRetornoService {
    calcularRetornoAnual(acao: Acao): number {
        return acao.quantidade * acao.dividendoAnual;
    }
}

class AcaoDescricaoService {
    descrever(acao: Acao): string {
        return `Ação: ${acao.nome}, Quantidade: ${acao.quantidade}, Preço por Ação: R$ ${acao.precoPorAcao}, Dividendo Anual: R$ ${acao.dividendoAnual}`;
    }
}

In [8]:
interface IRepository<T> {
    save(investimento: T): void;
}

class Repository<T> implements IRepository<T> {
    save(investimento: T): void {
        console.log(`Saving investment: ${JSON.stringify(investimento)}`);
    }
}

Registramos o Container de DI

In [9]:
class Container {
    private services = new Map<string, any>();

    register<T>(name: string, implementation: new (...args: any[]) => T): void {
        this.services.set(name, implementation);
    }

    resolve<T>(name: string): T {
        const implementation = this.services.get(name);
        if (!implementation) {
            throw new Error(`Service not found: ${name}`);
        }
        return new implementation();
    }
}

In [10]:
const container = new Container();
container.register<IRepository<Acao>>("AcaoRepository", Repository);
container.register<AcaoRetornoService>("AcaoRetornoService", AcaoRetornoService);
container.register<AcaoDescricaoService>("AcaoDescricaoService", AcaoDescricaoService);

Implementação

In [11]:
const acaoRepository = container.resolve<IRepository<Acao>>("AcaoRepository");
const acaoRetornoService = container.resolve<AcaoRetornoService>("AcaoRetornoService");
const acaoDescricaoService = container.resolve<AcaoDescricaoService>("AcaoDescricaoService");

const acao = new Acao("Empresa X", 5000, 100, 50, 5);

console.log(acaoDescricaoService.descrever(acao));

Ação: Empresa X, Quantidade: 100, Preço por Ação: R$ 50, Dividendo Anual: R$ 5


In [12]:
console.log(`Retorno anual: R$ ${acaoRetornoService.calcularRetornoAnual(acao)}`);

Retorno anual: R$ 500


In [13]:
acaoRepository.save(acao);

Saving investment: {"nome":"Empresa X","valorInvestido":5000,"quantidade":100,"precoPorAcao":50,"dividendoAnual":5}
